In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import mne
import pickle
from scipy.fft import fft, fftfreq, ifft
import seaborn as sns
import pandas as pd
from matplotlib import animation

%matplotlib widget
plt.rcParams['figure.figsize'] = [9, 8]

In [2]:
# Load meg channel config
dataset_path = os.path.join('cichy_data', 'subj0', 'MEG2_subj01_sess01_tsss_mc-3.fif')
raw = mne.io.read_raw_fif(dataset_path, preload=True)
chn_type = 'mag'
raw = raw.pick(chn_type)

Opening raw data file cichy_data/subj0/MEG2_subj01_sess01_tsss_mc-3.fif...
    Range : 4930000 ... 5162999 =   4930.000 ...  5162.999 secs
Ready.
Reading 0 ... 232999  =      0.000 ...   232.999 secs...


/var/folders/nv/wjmf18wd5_j38vg9v0cthl5h0000gn/T/ipykernel_5508/1878434330.py:3: RuntimeWarning: This filename (cichy_data/subj0/MEG2_subj01_sess01_tsss_mc-3.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(dataset_path, preload=True)


In [3]:
# PFI_freq
path = os.path.join('..', 'results', 'cichy_epoched', 'all_noshuffle_wavenetclass_semb10_drop0.4', 'PFIfreq',
                    'val_loss_PFIfreqs_ch4.npy')
pfi = np.load(open(path, 'rb'))
pfi = pfi[0, 0, 0] - pfi[0, :, 1:]

In [4]:
# times array
xf = fftfreq(256, 1/250)[:256//2]
xf = xf[3:126]
xf = np.round(xf)
xf = np.array([xf for _ in range(pfi.shape[1])]).T
xf = xf.reshape(-1)

pfi_pd = pfi

# magnitudes for color hues
mags = np.abs(np.mean(pfi, axis=0))
mags = np.array([mags/np.max(mags[:-1]) for _ in range(pfi_pd.shape[0])])
mags = mags.reshape(-1)

pfi_pd = pfi_pd.reshape(-1)

In [5]:
# put everything in a pd dataframe
pd_dict = {'Accuracy loss': pfi_pd, 'Frequency (Hz)': xf, 'relative magnitude': mags}
pfi_pd = pd.DataFrame(pd_dict)

In [10]:
%matplotlib widget
pfi_plot = sns.relplot(
    data=pfi_pd, kind="line",
    x="Frequency (Hz)", y="Accuracy loss", hue='relative magnitude', legend=None, palette='Reds', aspect=1.5
)

plt.savefig('/Users/ricsi/GoogleDrive/Projektek/The Brain Project/oxford/writings/neurips_figures/spatiospectral.pdf', format='pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
%matplotlib widget
evoked_ch1 = mne.EvokedArray(np.mean(pfi, axis=0, keepdims=True).T, raw.info, tmin=0)
evoked_ch1.plot_topomap(times=[0], ch_type='mag', time_unit='ms', scalings=1, units='Accuracy loss', time_format='', vmin=0)
plt.savefig('/Users/ricsi/GoogleDrive/Projektek/The Brain Project/oxford/writings/neurips_figures/spatiospectral_head.pdf', format='pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
# load pfi data
path = os.path.join('..', 'results', 'cichy_epoched', 'all_noshuffle_wavenetclass_semb10_drop0.4', 'PFIfreq',
                    'val_loss_PFIfreqs4_zeroed.npy')
pfi = np.load(open(path, 'rb'))
pfi = pfi[0, 0] - pfi[:, 1:]

Opening raw data file cichy_data/subj0/MEG2_subj01_sess01_tsss_mc-3.fif...
    Range : 4930000 ... 5162999 =   4930.000 ...  5162.999 secs
Ready.
Reading 0 ... 232999  =      0.000 ...   232.999 secs...


/var/folders/nv/wjmf18wd5_j38vg9v0cthl5h0000gn/T/ipykernel_4859/3329983199.py:3: RuntimeWarning: This filename (cichy_data/subj0/MEG2_subj01_sess01_tsss_mc-3.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(dataset_path, preload=True)


In [5]:
df = pd.DataFrame(pfi)
df = df.stack().reset_index()
df = df.rename(columns={'level_0': 'perm', 'level_1': 'frequency', 0: 'Accuracy loss'})

In [47]:
df

,perm,frequency,Accuracy loss
0,0,0,0.239859
1,0,1,0.212150
2,0,2,0.217673
3,0,3,0.211579
4,0,4,0.211579
...,...,...,...
620,4,120,0.002857
621,4,121,0.001428
622,4,122,-0.001714
623,4,123,-0.000667


In [9]:
pfi_plot = sns.relplot(
    data=df, kind="line",
    x="frequency", y="Accuracy loss", n_boot=1000, legend='brief', aspect=2, ci=95
)
plt.xticks(list(range(123))[::10], xf[::10])
a= 0

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
pfi_plot = sns.relplot(
    data=df, kind="line",
    x="frequency", y="Accuracy loss", n_boot=1000, legend='brief', aspect=2, ci=95
)
plt.xticks(list(range(123))[::10], xf[::10])
a= 0

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
# look at fourier transform
path = os.path.join('..', 'results', 'cichy_epoched', 'all_noshuffle_wavenetclass_semb10_drop0.4', 'PFIfreq',
                    'val_loss_PFIfreqs2_samples_zeroed.npy')
samples = np.load(open(path, 'rb'))

In [10]:
%matplotlib widget
plt.plot(np.abs(samples[0, :]))
plt.plot(np.abs(samples[50, :]))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [59]:
samples[0, -1:0]

array([], dtype=complex64)